## Validation of model predictions for carbon utilization

In [1]:
import cobra
import pandas as pd
import os

pd.set_option('display.max_rows', None)

# Call the metabolic models
s12 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')
kt2440 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iJN1463.xml')

# Open the PM experimental result file
pm = pd.read_excel(os.getcwd()+'\\input\\PM_result.xlsx')

In [2]:
# Function that compares experimental and simulation results
def check (x, y):
    if x == '+':
        if y == '+':
            ans = 'TP'
        else:
            ans = 'FN'
    else:
        if y == '+':
            ans = 'FP'
        else:
            ans = 'TN'
            
    return ans

# Function that simulate the growh of model in each sole carbon source
def growth_simulation (model):

    metabolites = pm['metabolites id'].values.tolist()
    exp_result = pm['s12 growth'].values.tolist()
    
    simulation_result = []
    min_growth = model.slim_optimize()*0.05

    for i in range(len(metabolites)):
        m = metabolites[i]
        x = exp_result[i]
        
        if m == 'x':
            y = '-'
        else:
            with model:
                m_id = 'EX_'+m+'_e'
            
                model.reactions.get_by_id('EX_glc__D_e').bounds = (0,0)
                
                try:
                    model.reactions.get_by_id(m_id).lower_bound = -6
            
                    simulation_growth = model.slim_optimize()
            
                    if simulation_growth > min_growth:
                        y = '+'
                    else:
                        y = '-'
                    
                except:
                    y = '-'
                    
                    
        simulation_result.append(check (x, y))
        
                    
    return simulation_result

In [3]:
# Run the functions
for gem in ['s12','kt2440']:
    model = globals()[gem]
    simulation_result = growth_simulation(model)
    pm[gem+' simulation result'] = simulation_result

In [4]:
# Print the result

data = {'TP':[],'TN':[],'FP':[],'FN':[]}

data['TP'].append(pm['s12 simulation result'].to_list().count('TP'))
data['TN'].append(pm['s12 simulation result'].to_list().count('TN'))
data['FP'].append(pm['s12 simulation result'].to_list().count('FP'))
data['FN'].append(pm['s12 simulation result'].to_list().count('FN'))

data['TP'].append(pm['kt2440 simulation result'].to_list().count('TP'))
data['TN'].append(pm['kt2440 simulation result'].to_list().count('TN'))
data['FP'].append(pm['kt2440 simulation result'].to_list().count('FP'))
data['FN'].append(pm['kt2440 simulation result'].to_list().count('FN'))

display(pd.DataFrame(data,index=['s12','kt2440']))
print('iSH1474 accuracy:',(pm[['s12 simulation result']].value_counts().TP + pm[['s12 simulation result']].value_counts().TN)/203 * 100)
print('iJN1462 accuracy:',(pm[['kt2440 simulation result']].value_counts().TP + pm[['kt2440 simulation result']].value_counts().TN)/203 * 100)

,TP,TN,FP,FN
s12,54,142,0,7
kt2440,48,129,13,13


iSH1474 accuracy: 96.55172413793103
iJN1462 accuracy: 87.192118226601


In [5]:
pm[pm['s12 simulation result']=='TN']

,PM test metabolites,metabolites id,s12 growth,s12 simulation result,kt2440 simulation result
0,A02 (L-Arabinose),arab__L,-,TN,TN
1,A03 (N-Acetyl-D-Glucosamine),acgam,-,TN,TN
4,A06 (D-Galactose),gal,-,TN,TN
8,A10 (D-Trehalose),tre,-,TN,TN
10,A12 (Dulcitol),galt,-,TN,TN
12,B02 (D-Sorbitol),sbt__D,-,TN,TN
14,B04 (L-Fucose),fuc__L,-,TN,TN
17,"B07 (D,L-α-Glycerol-Phosphate)",glyc3p,-,TN,TN
18,B08 (D-Xylose),xyl__D,-,TN,TN
20,B10 (Sodium Formate),for,-,TN,FP
